# 2020-02-03

Have a few duplicates coming up in similarity comparisons, therefore will see if there are still duplcates

Also trying out summing the similarity scores rather than purely the number of comments

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import pandas as pd
import numpy as np
import spacy
import json

In [2]:
insight_dir = 'C:\\Users\\Ronald Maj\\Documents\\GitHub\\InsightDataProject\\'

chan_info_df = pd.read_csv(insight_dir + 'data\\cleaned\\all_channels_info.csv', index_col=0)
vids_df = pd.read_csv(insight_dir+'data\\cleaned\\all_videos_dup_na_clean_df.csv', index_col=0)
comms_df = pd.read_csv(insight_dir+'data\\cleaned\\all_comments_dup_na_clean_df.csv', index_col=0)

C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Check if the comms_df is really duplicate free:
len(comms_df)

654394

In [4]:
comms_df[comms_df.duplicated('CommID')].head(10)

,CommID,authorChannelId,authorChannelUrl,authorDisplayName,authorProfileImageUrl,canRate,likeCount,moderationStatus,parentId,publishedAt,textDisplay,textOriginal,updatedAt,videoId,viewerRating
2175,Ugx4tPkUDK6SD6BfquJ4AaABAg,{'value': 'UCN5yuBsYaTyqH1eebwsiDTg'},http://www.youtube.com/channel/UCN5yuBsYaTyqH1...,Adam,https://yt3.ggpht.com/a/AGF-l78Er8kO5QSBQNMQpY...,True,103,NaN,0,2020-01-21T11:24:57.000Z,"Did I just watch two brothers legit have a ""to...","Did I just watch two brothers legit have a ""to...",2020-01-21T12:45:22.000Z,-TP0FuN5Saw,none
2179,Ugx4tPkUDK6SD6BfquJ4AaABAg.942cFkZBu4O942tUZ5yyus,{'value': 'UC31J-GnK6dQY0p2orKnMN_Q'},http://www.youtube.com/channel/UC31J-GnK6dQY0p...,Shammi Vlogs,https://yt3.ggpht.com/a/AGF-l7_1Zvd6j616fP_RnV...,True,9,NaN,Ugx4tPkUDK6SD6BfquJ4AaABAg,2020-01-21T13:55:31.000Z,Bahahahahaha,Bahahahahaha,2020-01-21T13:55:31.000Z,-TP0FuN5Saw,none
2180,Ugxb40ztxg3_E1NhNW54AaABAg,{'value': 'UC4V8j0LqVje8PcMX1--kODA'},http://www.youtube.com/channel/UC4V8j0LqVje8Pc...,Harrison Fraser,https://yt3.ggpht.com/a/AGF-l7_rvfIAZj2jOMzFaU...,True,97,NaN,0,2020-01-21T10:16:18.000Z,That fart was lethal,That fart was lethal,2020-01-21T10:16:18.000Z,-TP0FuN5Saw,none
2181,Ugxb40ztxg3_E1NhNW54AaABAg.942VOxopxbi9443Lj449vL,{'value': 'UC4V8j0LqVje8PcMX1--kODA'},http://www.youtube.com/channel/UC4V8j0LqVje8Pc...,Harrison Fraser,https://yt3.ggpht.com/a/AGF-l7_rvfIAZj2jOMzFaU...,True,1,NaN,Ugxb40ztxg3_E1NhNW54AaABAg,2020-01-22T00:49:40.000Z,@Bailey Burgess Fishing this is a fact,@Bailey Burgess Fishing this is a fact,2020-01-22T00:49:40.000Z,-TP0FuN5Saw,none
2182,Ugxb40ztxg3_E1NhNW54AaABAg.942VOxopxbi94435fK2Cyi,{'value': 'UCxpba6AJANtwPGBPLESd4XQ'},http://www.youtube.com/channel/UCxpba6AJANtwPG...,Bailey Burgess Fishing,https://yt3.ggpht.com/a/AGF-l79MnjAPIXogY76JI5...,True,2,NaN,Ugxb40ztxg3_E1NhNW54AaABAg,2020-01-22T00:47:28.000Z,I think I smelt it,I think I smelt it,2020-01-22T00:47:28.000Z,-TP0FuN5Saw,none
2183,Ugxb40ztxg3_E1NhNW54AaABAg.942VOxopxbi942t_BFesvY,{'value': 'UC31J-GnK6dQY0p2orKnMN_Q'},http://www.youtube.com/channel/UC31J-GnK6dQY0p...,Shammi Vlogs,https://yt3.ggpht.com/a/AGF-l7_1Zvd6j616fP_RnV...,True,11,NaN,Ugxb40ztxg3_E1NhNW54AaABAg,2020-01-21T13:56:17.000Z,Won me the point 😂,Won me the point 😂,2020-01-21T13:56:17.000Z,-TP0FuN5Saw,none
2184,UgwBbFZfhi8BXKTO3ah4AaABAg,{'value': 'UCcoYc-7x5KrhBVVSoUZf5aw'},http://www.youtube.com/channel/UCcoYc-7x5KrhBV...,FANNYLEROUX,https://yt3.ggpht.com/a/AGF-l79SMOSHnnrBf31x4G...,True,82,NaN,0,2020-01-21T10:19:52.000Z,"""where do you think you got that from"".\n\nSha...","""where do you think you got that from"".\n\nSha...",2020-01-21T10:19:52.000Z,-TP0FuN5Saw,none
2185,UgwCNXZKR3XwZ7aRqUZ4AaABAg,{'value': 'UCGWKYkHONw2y6LjVvFKbI3Q'},http://www.youtube.com/channel/UCGWKYkHONw2y6L...,Lachie W,https://yt3.ggpht.com/a/AGF-l7_ts9fDzAEtPfJjVk...,True,157,NaN,0,2020-01-21T11:01:56.000Z,Well I didn’t see that ending coming 😳,Well I didn’t see that ending coming 😳,2020-01-21T11:01:56.000Z,-TP0FuN5Saw,none
2186,UgwCNXZKR3XwZ7aRqUZ4AaABAg.942_cBLwvEL942tQTP36MV,{'value': 'UC31J-GnK6dQY0p2orKnMN_Q'},http://www.youtube.com/channel/UC31J-GnK6dQY0p...,Shammi Vlogs,https://yt3.ggpht.com/a/AGF-l7_1Zvd6j616fP_RnV...,True,27,NaN,UgwCNXZKR3XwZ7aRqUZ4AaABAg,2020-01-21T13:54:57.000Z,At least it was a Happy Ending 😊,At least it was a Happy Ending 😊,2020-01-21T13:54:57.000Z,-TP0FuN5Saw,none
2187,UgzyPKmpwKZEfBQY_f94AaABAg,{'value': 'UCWDkgjUWxkdMiop8hYuLlsA'},http://www.youtube.com/channel/UCWDkgjUWxkdMio...,josh tulk,https://yt3.ggpht.com/a/AGF-l7_lWm_8i3804dbxC0...,True,83,NaN,0,2020-01-21T10:01:31.000Z,Brothers just making memories 😂😂,Brothers just making memories 😂😂,2020-01-21T10:01:31.000Z,-TP0FuN5Saw,none


Looks like there are still duplicates if we search on CommID. Need to remove these and save:

In [5]:
len(comms_df)

654394

In [6]:
comms_df.drop_duplicates(subset='CommID', inplace=True)

In [7]:
len(comms_df)

654073

In [8]:
comms_df[comms_df.duplicated('CommID')].head(10)

,CommID,authorChannelId,authorChannelUrl,authorDisplayName,authorProfileImageUrl,canRate,likeCount,moderationStatus,parentId,publishedAt,textDisplay,textOriginal,updatedAt,videoId,viewerRating


In [9]:
#comms_df.to_csv(insight_dir+'data\\cleaned\\all_comments_dup_na_clean_new_df.csv')

For now the comment matrix is built upon the previous .csv file, and therefore I will need to use this for now.

However, I will re-run the calculation of the matrix once more once I have time.

In [10]:
comms_df = pd.read_csv(insight_dir+'data\\cleaned\\all_comments_dup_na_clean_df.csv', index_col=0)

Continuing on with producing new metric:

In [11]:
nlp = spacy.load("en_core_web_lg")

In [12]:
database_mat = np.load('C:\\Users\\Ronald Maj\\Documents\\GitHub\\InsightDataProject\\data\\cleaned\\comms_mat_all.npy')

In [13]:
# Turning large numbers into 'human' form
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [15]:
(search_term, sim_thresh) = ('canon', 0.55)

In [16]:
#### Put in search term of interest:

search_doc = nlp(search_term)

# Get the vector representation:
search_vec = search_doc.vector.reshape(1,-1)

#### Create similarity vector
sim_vec = cosine_similarity(search_vec,database_mat)

####  Return ordered list of comments based on threshold:
tf_ar = sim_vec > sim_thresh

# Create a results_info_df that collects all the relevant information on 
# the resultant list produced:
results_info_df = comms_df[tf_ar[0]][['CommID',
                                           'authorChannelUrl',
                                           'authorDisplayName',
                                           'authorProfileImageUrl',
                                           'parentId',
                                           'publishedAt',
                                           'textDisplay',
                                           'videoId']].copy(deep=True)

# Assign a new column in the results_info dataframe to associate the similarity score with the correct comment and sort:
results_info_df['sim_score'] = sim_vec[0][tf_ar[0]]
results_info_df.sort_values(by='sim_score',ascending=False, inplace=True)

#### Need to rename the column with the video ID to match up with that in the vids_df
cols = list(results_info_df.columns)
cols[-2] = 'VidID'
results_info_df.columns = cols
results_info_df = results_info_df.merge(
    right=vids_df,
    how='left',
    on='VidID',
    suffixes=('_comm', '_vid'))

#### Lastly get the channel info:

# Rename common columns:
cols = list(results_info_df.columns)    
cols[5] = 'publishedAt_comm'    
results_info_df.columns = cols    
results_info_df = results_info_df.merge(
    right=chan_info_df,
    how='left',
    on='ChannelID',
    suffixes=('_vid', '_chan'))

comm_count = Counter(results_info_df['ChannelID'])
comm_count_list = comm_count.most_common()

chan_dict_list= [human_format(len(comm_count))]

## Try adding the sum of the similarity scores, rather than the number of comments for that channel

#counts_list = {}
#for value in L:
#    if value[0] in counts:
#        counts[value[0]] += value[1]
#    else:
#        counts[value[0]] = value[1]



for chan_id,count in comm_count_list[0:6]:
    chan_result = results_info_df[results_info_df['ChannelID'] == chan_id].iloc[0]
    thumb_dict_str = chan_result['thumbnails']
    thumb_dict_str = thumb_dict_str.replace("\'", "\"")
    thumb_dict = json.loads(thumb_dict_str)
    prof_pic_url = thumb_dict['default']['url']
    chan_result_dict = {
        'Channel Name':chan_result['title'],
        'Chan_url':'https://www.youtube.com/channel/'+chan_id,
        'Comment Fraction':human_format(count),
        'No. Subscribers':human_format(chan_result['subscriberCount']),
        'No. Views':human_format(chan_result['viewCount_chan']),
        'Likes/Views (on video)':round(chan_result['likeCount'] / chan_result['viewCount_vid'],5),
        'Comments/Views (on video)':round(chan_result['commentCount_vid'] / chan_result['viewCount_vid'],5),
        'Chan Profile Pic':prof_pic_url
        }
    chan_dict_list.append(chan_result_dict)

comm_dict_list= [len(results_info_df)]

for i in results_info_df[0:6].index:
    comm_result_dict = {
    'Name':results_info_df.iloc[i]['authorDisplayName'],
    'Comment':results_info_df.iloc[i]['textDisplay'],
    'Sim Score':round(results_info_df.iloc[i]['sim_score'],2),
    'Profile Pic':results_info_df.iloc[i]['authorProfileImageUrl'],
    'Vid_url':'https://www.youtube.com/watch?v='+results_info_df.iloc[i]['VidID'],
    'Vid_title':results_info_df.iloc[i]['VidTitle'],
    'Channel Name':results_info_df.iloc[i]['title'],
    'Chan_url':'https://www.youtube.com/channel/'+results_info_df.iloc[i]['ChannelID']        
        }
    comm_dict_list.append(comm_result_dict)

In [17]:
sim_vec.shape

(1, 654394)

In [18]:
comms_df['sim_score'] = np.transpose(sim_vec)

In [19]:
comms_df.head(10)

,CommID,authorChannelId,authorChannelUrl,authorDisplayName,authorProfileImageUrl,canRate,likeCount,moderationStatus,parentId,publishedAt,textDisplay,textOriginal,updatedAt,videoId,viewerRating,sim_score
0,Ugx4tPkUDK6SD6BfquJ4AaABAg,{'value': 'UCN5yuBsYaTyqH1eebwsiDTg'},http://www.youtube.com/channel/UCN5yuBsYaTyqH1...,Adam,https://yt3.ggpht.com/a/AGF-l78Er8kO5QSBQNMQpY...,True,90,NaN,0,2020-01-21T11:24:57.000Z,"Did I just watch two brothers legit have a ""to...","Did I just watch two brothers legit have a ""to...",2020-01-21T12:45:22.000Z,-TP0FuN5Saw,none,0.228453
1,Ugx4tPkUDK6SD6BfquJ4AaABAg.942cFkZBu4O9442vbfk0Zx,{'value': 'UCIO8eoOzwAmWRwudrYn1Sew'},http://www.youtube.com/channel/UCIO8eoOzwAmWRw...,Synical,https://yt3.ggpht.com/a/AGF-l7-wr3mhVx6G5DxbGy...,True,0,NaN,Ugx4tPkUDK6SD6BfquJ4AaABAg,2020-01-22T00:45:58.000Z,"@Shammi Vlogs I thought that was a joke, lol","@Shammi Vlogs I thought that was a joke, lol",2020-01-22T00:45:58.000Z,-TP0FuN5Saw,none,0.230124
2,Ugx4tPkUDK6SD6BfquJ4AaABAg.942cFkZBu4O942tUZ5yyus,{'value': 'UC31J-GnK6dQY0p2orKnMN_Q'},http://www.youtube.com/channel/UC31J-GnK6dQY0p...,Shammi Vlogs,https://yt3.ggpht.com/a/AGF-l7_1Zvd6j616fP_RnV...,True,7,NaN,Ugx4tPkUDK6SD6BfquJ4AaABAg,2020-01-21T13:55:31.000Z,Bahahahahaha,Bahahahahaha,2020-01-21T13:55:31.000Z,-TP0FuN5Saw,none,-0.054987
3,Ugx4tPkUDK6SD6BfquJ4AaABAg.942cFkZBu4O942lQp3-bgX,{'value': 'UCN5yuBsYaTyqH1eebwsiDTg'},http://www.youtube.com/channel/UCN5yuBsYaTyqH1...,Adam,https://yt3.ggpht.com/a/AGF-l78Er8kO5QSBQNMQpY...,True,3,NaN,Ugx4tPkUDK6SD6BfquJ4AaABAg,2020-01-21T12:45:06.000Z,@Larry lmao true \n\nPs \nIll edit it,@Larry lmao true \n\nPs \nIll edit it,2020-01-21T12:45:06.000Z,-TP0FuN5Saw,none,0.294484
4,Ugx4tPkUDK6SD6BfquJ4AaABAg.942cFkZBu4O942kC_rsj3B,{'value': 'UClx1z4juTQWqhcYiTCEV3bA'},http://www.youtube.com/channel/UClx1z4juTQWqhc...,Larry,https://yt3.ggpht.com/a/AGF-l79NxTgxxVsBDnhhPS...,True,3,NaN,Ugx4tPkUDK6SD6BfquJ4AaABAg,2020-01-21T12:34:25.000Z,Adam brothers* and the worst part about it is ...,Adam brothers* and the worst part about it is ...,2020-01-21T12:34:25.000Z,-TP0FuN5Saw,none,0.251673
5,Ugxb40ztxg3_E1NhNW54AaABAg,{'value': 'UC4V8j0LqVje8PcMX1--kODA'},http://www.youtube.com/channel/UC4V8j0LqVje8Pc...,Harrison Fraser,https://yt3.ggpht.com/a/AGF-l7_rvfIAZj2jOMzFaU...,True,82,NaN,0,2020-01-21T10:16:18.000Z,That fart was lethal,That fart was lethal,2020-01-21T10:16:18.000Z,-TP0FuN5Saw,none,0.196072
6,Ugxb40ztxg3_E1NhNW54AaABAg.942VOxopxbi9443Lj449vL,{'value': 'UC4V8j0LqVje8PcMX1--kODA'},http://www.youtube.com/channel/UC4V8j0LqVje8Pc...,Harrison Fraser,https://yt3.ggpht.com/a/AGF-l7_rvfIAZj2jOMzFaU...,True,0,NaN,Ugxb40ztxg3_E1NhNW54AaABAg,2020-01-22T00:49:40.000Z,@Bailey Burgess Fishing this is a fact,@Bailey Burgess Fishing this is a fact,2020-01-22T00:49:40.000Z,-TP0FuN5Saw,none,0.277104
7,Ugxb40ztxg3_E1NhNW54AaABAg.942VOxopxbi94435fK2Cyi,{'value': 'UCxpba6AJANtwPGBPLESd4XQ'},http://www.youtube.com/channel/UCxpba6AJANtwPG...,Bailey Burgess Fishing,https://yt3.ggpht.com/a/AGF-l79MnjAPIXogY76JI5...,True,1,NaN,Ugxb40ztxg3_E1NhNW54AaABAg,2020-01-22T00:47:28.000Z,I think I smelt it,I think I smelt it,2020-01-22T00:47:28.000Z,-TP0FuN5Saw,none,0.225088
8,Ugxb40ztxg3_E1NhNW54AaABAg.942VOxopxbi942t_BFesvY,{'value': 'UC31J-GnK6dQY0p2orKnMN_Q'},http://www.youtube.com/channel/UC31J-GnK6dQY0p...,Shammi Vlogs,https://yt3.ggpht.com/a/AGF-l7_1Zvd6j616fP_RnV...,True,9,NaN,Ugxb40ztxg3_E1NhNW54AaABAg,2020-01-21T13:56:17.000Z,Won me the point 😂,Won me the point 😂,2020-01-21T13:56:17.000Z,-TP0FuN5Saw,none,0.219578
9,UgwCNXZKR3XwZ7aRqUZ4AaABAg,{'value': 'UCGWKYkHONw2y6LjVvFKbI3Q'},http://www.youtube.com/channel/UCGWKYkHONw2y6L...,Lachie W,https://yt3.ggpht.com/a/AGF-l7_ts9fDzAEtPfJjVk...,True,150,NaN,0,2020-01-21T11:01:56.000Z,Well I didn’t see that ending coming 😳,Well I didn’t see that ending coming 😳,2020-01-21T11:01:56.000Z,-TP0FuN5Saw,none,0.249949


In [20]:
comms_df.sort_values(by='sim_score',ascending=False,inplace=True)

In [21]:
comms_df.head(50)

,CommID,authorChannelId,authorChannelUrl,authorDisplayName,authorProfileImageUrl,canRate,likeCount,moderationStatus,parentId,publishedAt,textDisplay,textOriginal,updatedAt,videoId,viewerRating,sim_score
768500,UgxJrhYrKTVKx8INrIN4AaABAg.8zzzEW68-lS9-XmQsuPckZ,{'value': 'UCj6rni8SDHEE4HaybZQKVEQ'},http://www.youtube.com/channel/UCj6rni8SDHEE4H...,Emily,https://yt3.ggpht.com/a/AGF-l79UChkMVbfJUFe3vN...,True,0,NaN,UgxJrhYrKTVKx8INrIN4AaABAg,2019-10-01T07:13:20.000Z,Canon G7X ❤️,Canon G7X ❤️,2019-10-01T07:13:20.000Z,VKAntiEQ3j0,none,1.000000
60106,UgzeeoQ81TECqc47ZHp4AaABAg.9-aHxIIjeeR93lu7SXC0dB,{'value': 'UCiBUOOlQGXKXgE93RQNOqbw'},http://www.youtube.com/channel/UCiBUOOlQGXKXgE...,Jerimiah Cook Vlogs,https://yt3.ggpht.com/a/AGF-l79WjIogXbsQZEmYK4...,True,0,NaN,UgzeeoQ81TECqc47ZHp4AaABAg,2020-01-14T14:14:46.000Z,and Canon,and Canon,2020-01-14T14:14:46.000Z,xg05Jza995o,none,0.861038
56759,UgxG0hNO_jKY3MpwtvZ4AaABAg,{'value': 'UCXg4rrtzpTTopUoeoqTwwSQ'},http://www.youtube.com/channel/UCXg4rrtzpTTopU...,Matt Vivanco,https://yt3.ggpht.com/a/AGF-l7_q_Fp-bOQrsysVNn...,True,7,NaN,0,2019-12-02T21:51:54.000Z,Please review the Canon EOS RP!,Please review the Canon EOS RP!,2019-12-02T21:51:54.000Z,eKfpM4cBfwo,none,0.725374
55224,UgzblvrCbD_EF-yb1Zp4AaABAg,{'value': 'UC8Ofz3iJg1oPCidzujyO9TA'},http://www.youtube.com/channel/UC8Ofz3iJg1oPCi...,PC Gamer,https://yt3.ggpht.com/a/AGF-l7_g62CZ7C2pXe4EUW...,True,0,NaN,0,2019-12-30T21:23:02.000Z,Waiting for your Canon Eos RP review.,Waiting for your Canon Eos RP review.,2019-12-30T21:23:02.000Z,JhdoY-ckzx4,none,0.674260
51628,Ugyd63X8kafcMaPZ--B4AaABAg,{'value': 'UCiAVwn1k6_fCZchPeu2zOJA'},http://www.youtube.com/channel/UCiAVwn1k6_fCZc...,Chad G.,https://yt3.ggpht.com/a/AGF-l7__5Frycp3cEfDIuA...,True,0,NaN,0,2019-01-07T14:47:12.000Z,Is this canon?,Is this canon?,2019-01-07T14:47:12.000Z,P5tN1kXYQlY,none,0.657506
56710,Ugynl5rtz_ShnQZFrnN4AaABAg,{'value': 'UCgLFR7-st9PmcR48pJwtDsQ'},http://www.youtube.com/channel/UCgLFR7-st9PmcR...,Zacharias Hulthén,https://yt3.ggpht.com/a/AGF-l78dtYuaKV_Aw_D5Il...,True,0,NaN,0,2020-01-07T20:58:18.000Z,Yes plx Canon EOS RP and some good lens for s...,Yes plx Canon EOS RP and some good lens for s...,2020-01-07T20:58:18.000Z,eKfpM4cBfwo,none,0.656704
873578,UgwmT6_USDjuFy2LL1p4AaABAg,{'value': 'UCZr2O4MinSfAWKPWBk5Z2XA'},http://www.youtube.com/channel/UCZr2O4MinSfAWK...,KAMY J.,https://yt3.ggpht.com/a/AGF-l78UInnXQW8xo3WuVi...,True,2,NaN,0,2019-12-23T22:25:12.000Z,The canon looks better,The canon looks better,2019-12-23T22:25:12.000Z,O_PVav8Y0xc,none,0.630586
56606,UgwRt5Tzffa6gqCtwrV4AaABAg,{'value': 'UC9qrw7Qpoh7znHWZBW4HzKw'},http://www.youtube.com/channel/UC9qrw7Qpoh7znH...,Steve Murray,https://yt3.ggpht.com/a/AGF-l79Q81FJ999W3mc8rC...,True,213,NaN,0,2019-12-03T05:55:57.000Z,Yes canon eos rp field test would be appreciated,Yes canon eos rp field test would be appreciated,2019-12-03T05:55:57.000Z,eKfpM4cBfwo,none,0.621265
225998,UgyNLPv14FUCQjk0df94AaABAg.92r0pr6QRaO92r4DmuIKp1,{'value': 'UCgWfS_47YPVbKx5EK4FLm4A'},http://www.youtube.com/channel/UCgWfS_47YPVbKx...,Jenn Im,https://yt3.ggpht.com/a/AGF-l78dBoc7rLWB6tLgSz...,True,0,NaN,UgyNLPv14FUCQjk0df94AaABAg,2019-12-22T17:56:34.000Z,I brought my Canon M50!,I brought my Canon M50!,2019-12-22T17:56:34.000Z,ZcCp6ghTP78,none,0.611535
56585,UgzWK2UyVuzYVhHgti94AaABAg,{'value': 'UCIM4NXw1dvS3e3yQOjodnVQ'},http://www.youtube.com/channel/UCIM4NXw1dvS3e3...,Jabin,https://yt3.ggpht.com/a/AGF-l7-GzDyXbW24FHyUgr...,True,4,NaN,0,2019-12-24T11:31:30.000Z,I would like to watch the canon eos rp review,I would like to watch the canon eos rp review,2019-12-24T11:31:30.000Z,eKfpM4cBfwo,none,0.609891


Re-write the function to choose the number of comments rather than the similarity of comments:

In [22]:
(search_term, num_comms) = ('canon', 50)

In [110]:
comms_df = pd.read_csv(insight_dir+'data\\cleaned\\all_comments_dup_na_clean_df.csv', index_col=0)

C:\ProgramData\Anaconda3\envs\insight\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [111]:
comms_df.shape

(654394, 15)

In [112]:
#### Put in search term of interest:

search_doc = nlp(search_term)

# Get the vector representation:
search_vec = search_doc.vector.reshape(1,-1)

#### Create similarity vector
sim_vec = cosine_similarity(search_vec,database_mat)

# Assign the scores to the dataframe and sort by the similarity scores
comms_df['sim_score'] = (sim_vec[0])

rel_comms_df = comms_df.sort_values(by='sim_score',ascending=False).head(num_comms)

In [113]:
rel_comms_df.shape

(50, 16)

In [85]:
# Create a results_info_df that collects all the relevant information on 
# the resultant list produced:
results_info_df = rel_comms_df[[
                                'CommID',
                                'authorChannelUrl',
                                'authorDisplayName',
                                'authorProfileImageUrl',
                                'parentId',
                                'publishedAt',
                                'textDisplay',
                                'videoId',
                                'sim_score']].copy(deep=True)

In [86]:
results_info_df.head(5)

,CommID,authorChannelUrl,authorDisplayName,authorProfileImageUrl,parentId,publishedAt,textDisplay,videoId,sim_score
768500,UgxJrhYrKTVKx8INrIN4AaABAg.8zzzEW68-lS9-XmQsuPckZ,http://www.youtube.com/channel/UCj6rni8SDHEE4H...,Emily,https://yt3.ggpht.com/a/AGF-l79UChkMVbfJUFe3vN...,UgxJrhYrKTVKx8INrIN4AaABAg,2019-10-01T07:13:20.000Z,Canon G7X ❤️,VKAntiEQ3j0,1.000000
60106,UgzeeoQ81TECqc47ZHp4AaABAg.9-aHxIIjeeR93lu7SXC0dB,http://www.youtube.com/channel/UCiBUOOlQGXKXgE...,Jerimiah Cook Vlogs,https://yt3.ggpht.com/a/AGF-l79WjIogXbsQZEmYK4...,UgzeeoQ81TECqc47ZHp4AaABAg,2020-01-14T14:14:46.000Z,and Canon,xg05Jza995o,0.861038
56759,UgxG0hNO_jKY3MpwtvZ4AaABAg,http://www.youtube.com/channel/UCXg4rrtzpTTopU...,Matt Vivanco,https://yt3.ggpht.com/a/AGF-l7_q_Fp-bOQrsysVNn...,0,2019-12-02T21:51:54.000Z,Please review the Canon EOS RP!,eKfpM4cBfwo,0.725374
55224,UgzblvrCbD_EF-yb1Zp4AaABAg,http://www.youtube.com/channel/UC8Ofz3iJg1oPCi...,PC Gamer,https://yt3.ggpht.com/a/AGF-l7_g62CZ7C2pXe4EUW...,0,2019-12-30T21:23:02.000Z,Waiting for your Canon Eos RP review.,JhdoY-ckzx4,0.674260
51628,Ugyd63X8kafcMaPZ--B4AaABAg,http://www.youtube.com/channel/UCiAVwn1k6_fCZc...,Chad G.,https://yt3.ggpht.com/a/AGF-l7__5Frycp3cEfDIuA...,0,2019-01-07T14:47:12.000Z,Is this canon?,P5tN1kXYQlY,0.657506


In [87]:
len(results_info_df)

50

Figured out the problem of not assigning the sim_score correctly - just needed to refer to the first element [0]

In [88]:
#### Need to rename the column with the video ID to match up with that in the vids_df
cols = list(results_info_df.columns)

In [89]:
cols[-2]

'videoId'

In [90]:
len(results_info_df)

50

In [91]:
cols[-2] = 'VidID'
results_info_df.columns = cols

In [92]:
len(results_info_df)

50

In [97]:
results_info_df.duplicated(subset='CommID')

768500    False
60106     False
56759     False
55224     False
51628     False
56710     False
873578    False
56606     False
225998    False
56585     False
186789    False
56569     False
110301    False
58543     False
56735     False
56760     False
56551     False
58326     False
54776     False
56651     False
56379     False
56329     False
56707     False
873520    False
56584     False
61021     False
56639     False
873628    False
57620     False
22606     False
109252    False
54803     False
56548     False
56545     False
56762     False
54899     False
161069    False
56647     False
54843     False
56779     False
56708     False
70551     False
56761     False
782628    False
56737     False
894184    False
54872     False
54044     False
54799     False
56583     False
dtype: bool

In [98]:
len(results_info_df.duplicated(subset='CommID'))

50

In [99]:
results_info_df = results_info_df.merge(
    right=vids_df,
    how='left',
    on='VidID',
    suffixes=('_comm', '_vid'))

In [100]:
len(results_info_df)

52

In [102]:
results_info_df[results_info_df.duplicated(subset='CommID')]

,CommID,authorChannelUrl,authorDisplayName,authorProfileImageUrl,parentId,publishedAt,textDisplay,VidID,sim_score,ChannelID,VidTitle,VidDescription,VidPublished,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
11,UgwlsX9xZxu1FEYcFC14AaABAg.8uvRlDmY0H88uvptTGtGWO,http://www.youtube.com/channel/UCTImTQyWGg5OMs...,AyeAuto Vlog,https://yt3.ggpht.com/a/AGF-l793L5BOFqksv070iZ...,UgwlsX9xZxu1FEYcFC14AaABAg,2019-05-14T21:36:14.000Z,"Canon 600D, Samsung Note 9, Go Pro Hero 6",MddZU9VsdOo,0.602596,UCTImTQyWGg5OMs2BUiVI6nQ,TATA Harrier - The Most Detailed Malayalam Rev...,#TataHarrier #TestDriveReview #MalayalamHarrie...,2019-05-12T05:43:18.000Z,92637.0,1470.0,71.0,0.0,425.0
43,UgyhjRTefqEAaslVB5R4AaABAg,http://www.youtube.com/channel/UCFK32Yr6I2olVw...,AJ Masters,https://yt3.ggpht.com/a/AGF-l7-OE4TF0wLhfhsz58...,0,2019-12-10T04:48:32.000Z,the camera lense :( #dirtyaf,nTGM1FpoEEA,0.516690,UCyQb1TTrfRzQZmEfsx770qw,C´EST NORMAL - A TRUE FAMILY COMPANY! | VLOG⁴ 42,Get your C´est normal gear here: http://www.ce...,2019-12-08T18:15:37.000Z,507144.0,11058.0,308.0,0.0,657.0


In [103]:
results_info_df[results_info_df['CommID'] == 'UgyhjRTefqEAaslVB5R4AaABAg']

,CommID,authorChannelUrl,authorDisplayName,authorProfileImageUrl,parentId,publishedAt,textDisplay,VidID,sim_score,ChannelID,VidTitle,VidDescription,VidPublished,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
42,UgyhjRTefqEAaslVB5R4AaABAg,http://www.youtube.com/channel/UCFK32Yr6I2olVw...,AJ Masters,https://yt3.ggpht.com/a/AGF-l7-OE4TF0wLhfhsz58...,0,2019-12-10T04:48:32.000Z,the camera lense :( #dirtyaf,nTGM1FpoEEA,0.51669,UCyQb1TTrfRzQZmEfsx770qw,C´EST NORMAL - A TRUE FAMILY COMPANY! | VLOG⁴ 42,Get your C´est normal gear here: http://www.ce...,2019-12-08T18:15:37.000Z,505924.0,11049.0,307.0,0.0,657.0
43,UgyhjRTefqEAaslVB5R4AaABAg,http://www.youtube.com/channel/UCFK32Yr6I2olVw...,AJ Masters,https://yt3.ggpht.com/a/AGF-l7-OE4TF0wLhfhsz58...,0,2019-12-10T04:48:32.000Z,the camera lense :( #dirtyaf,nTGM1FpoEEA,0.51669,UCyQb1TTrfRzQZmEfsx770qw,C´EST NORMAL - A TRUE FAMILY COMPANY! | VLOG⁴ 42,Get your C´est normal gear here: http://www.ce...,2019-12-08T18:15:37.000Z,507144.0,11058.0,308.0,0.0,657.0


In [104]:
# Remove duplicated comments:
results_info_df.drop_duplicates(subset='CommID', inplace=True)

In [105]:
#### Lastly get the channel info:

# Rename common columns:
cols = list(results_info_df.columns)    
cols[5] = 'publishedAt_comm'    
results_info_df.columns = cols    
results_info_df = results_info_df.merge(
    right=chan_info_df,
    how='left',
    on='ChannelID',
    suffixes=('_vid', '_chan'))

comm_count = Counter(results_info_df['ChannelID'])
comm_count_list = comm_count.most_common()

chan_dict_list= [human_format(len(comm_count))]


# Send the relevant info to the website:

for chan_id,count in comm_count_list[0:6]:
    chan_result = results_info_df[results_info_df['ChannelID'] == chan_id].iloc[0]
    thumb_dict_str = chan_result['thumbnails']
    thumb_dict_str = thumb_dict_str.replace("\'", "\"")
    thumb_dict = json.loads(thumb_dict_str)
    prof_pic_url = thumb_dict['default']['url']
    chan_result_dict = {
        'Channel Name':chan_result['title'],
        'Chan_url':'https://www.youtube.com/channel/'+chan_id,
        'Comment Fraction':human_format(count),
        'No. Subscribers':human_format(chan_result['subscriberCount']),
        'No. Views':human_format(chan_result['viewCount_chan']),
        'Likes/Views (on video)':round(chan_result['likeCount'] / chan_result['viewCount_vid'],5),
        'Comments/Views (on video)':round(chan_result['commentCount_vid'] / chan_result['viewCount_vid'],5),
        'Chan Profile Pic':prof_pic_url
        }
    chan_dict_list.append(chan_result_dict)

comm_dict_list= [len(results_info_df)]

for i in results_info_df[0:6].index:
    comm_result_dict = {
    'Name':results_info_df.iloc[i]['authorDisplayName'],
    'Comment':results_info_df.iloc[i]['textDisplay'],
    'Sim Score':round(results_info_df.iloc[i]['sim_score'],2),
    'Profile Pic':results_info_df.iloc[i]['authorProfileImageUrl'],
    'Vid_url':'https://www.youtube.com/watch?v='+results_info_df.iloc[i]['VidID'],
    'Vid_title':results_info_df.iloc[i]['VidTitle'],
    'Channel Name':results_info_df.iloc[i]['title'],
    'Chan_url':'https://www.youtube.com/channel/'+results_info_df.iloc[i]['ChannelID']        
        }
    comm_dict_list.append(comm_result_dict)

In [106]:
# Check results
comm_dict_list

[50,
 {'Name': 'Emily',
  'Comment': 'Canon G7X ❤️',
  'Sim Score': 1.0,
  'Profile Pic': 'https://yt3.ggpht.com/a/AGF-l79UChkMVbfJUFe3vNYgAcS2nzcf4mxFh-mXWA=s48-c-k-c0xffffffff-no-rj-mo',
  'Vid_url': 'https://www.youtube.com/watch?v=VKAntiEQ3j0',
  'Vid_title': 'AUTUMN OUTFITS HAUL 2019 🍁 | HUGE MISSGUIDED TRY ON + DISCOUNT CODE!!  👀 ad',
  'Channel Name': 'Danielle Mansutti',
  'Chan_url': 'https://www.youtube.com/channel/UCJRFJl5UDs4Citzs-uRcrxQ'},
 {'Name': 'Jerimiah Cook Vlogs',
  'Comment': 'and Canon',
  'Sim Score': 0.86,
  'Profile Pic': 'https://yt3.ggpht.com/a/AGF-l79WjIogXbsQZEmYK49P5CJWfBtXhvMghZzhrw=s48-c-k-c0xffffffff-no-rj-mo',
  'Vid_url': 'https://www.youtube.com/watch?v=xg05Jza995o',
  'Vid_title': 'How I Film a Cinematic Sequence - Behind The Scenes',
  'Channel Name': 'Peter McKinnon',
  'Chan_url': 'https://www.youtube.com/channel/UC3DkFux8Iv-aYnTRWzwaiBA'},
 {'Name': 'Matt Vivanco',
  'Comment': 'Please review the Canon EOS RP!',
  'Sim Score': 0.73,
  'Profile 

In [107]:
# Wrong number of comments
num_comms

50

In [108]:
len(results_info_df)

50

In [109]:
chan_dict_list

['12',
 {'Channel Name': 'Peter McKinnon',
  'Chan_url': 'https://www.youtube.com/channel/UC3DkFux8Iv-aYnTRWzwaiBA',
  'Comment Fraction': '36',
  'No. Subscribers': '4.27M',
  'No. Views': '300M',
  'Likes/Views (on video)': 0.0503,
  'Comments/Views (on video)': 0.00312,
  'Chan Profile Pic': 'https://yt3.ggpht.com/a/AGF-l7_zHtm1Mcchr3Zq7q518oPcTJh951-wRp9GHQ=s88-c-k-c0xffffffff-no-rj-mo'},
 {'Channel Name': 'Sarah Rae Vlogas',
  'Chan_url': 'https://www.youtube.com/channel/UC0YgscPJVsW-4tIx2U499cw',
  'Comment Fraction': '3',
  'No. Subscribers': '252K',
  'No. Views': '36.2M',
  'Likes/Views (on video)': 0.03935,
  'Comments/Views (on video)': 0.00395,
  'Chan Profile Pic': 'https://yt3.ggpht.com/a/AGF-l7-lBZPgpr7QsaBQELzS5Rz62fC7Vr3rVKqKww=s88-c-k-c0xffffffff-no-rj-mo'},
 {'Channel Name': 'joanday',
  'Chan_url': 'https://www.youtube.com/channel/UCAo41QETLlwvJ-oqjmEPt8A',
  'Comment Fraction': '2',
  'No. Subscribers': '397K',
  'No. Views': '51.9M',
  'Likes/Views (on video)': 0.

Bring it all together

In [114]:
# Turning large numbers into 'human' form
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [115]:
def get_comment_channel_results(search_term, num_comms):
    #### Put in search term of interest:
    search_doc = nlp(search_term)
    
    # Get the vector representation:
    search_vec = search_doc.vector.reshape(1,-1)
    
    #### Create similarity vector
    sim_vec = cosine_similarity(search_vec,database_mat)
    
    # Assign the scores to the dataframe and sort by the similarity scores
    comms_df['sim_score'] = (sim_vec[0])

    rel_comms_df = comms_df.sort_values(by='sim_score',ascending=False).head(num_comms)
     
    # Create a results_info_df that collects all the relevant information on 
    # the resultant df:
    results_info_df = rel_comms_df[[
                                    'CommID',
                                    'authorChannelUrl',
                                    'authorDisplayName',
                                    'authorProfileImageUrl',
                                    'parentId',
                                    'publishedAt',
                                    'textDisplay',
                                    'videoId',
                                    'sim_score']].copy(deep=True)
    
    #### Need to rename the column with the video ID to match up with that in the vids_df
    cols = list(results_info_df.columns)
    cols[-2] = 'VidID'
    results_info_df.columns = cols
    results_info_df = results_info_df.merge(
        right=vids_df,
        how='left',
        on='VidID',
        suffixes=('_comm', '_vid'))
    
    # Remove duplicated comments:
    results_info_df.drop_duplicates(subset='CommID', inplace=True)
    
    #### Lastly get the channel info:
    
    # Rename common columns:
    cols = list(results_info_df.columns)    
    cols[5] = 'publishedAt_comm'    
    results_info_df.columns = cols    
    results_info_df = results_info_df.merge(
        right=chan_info_df,
        how='left',
        on='ChannelID',
        suffixes=('_vid', '_chan'))
    
    # Remove duplicated comments:
    results_info_df.drop_duplicates(subset='CommID', inplace=True)
    
    comm_count = Counter(results_info_df['ChannelID'])
    comm_count_list = comm_count.most_common()
    
    chan_dict_list= [human_format(len(comm_count))]
    
    # Send the relevant info to the website:

    for chan_id,count in comm_count_list[0:6]:
        chan_result = results_info_df[results_info_df['ChannelID'] == chan_id].iloc[0]
        thumb_dict_str = chan_result['thumbnails']
        thumb_dict_str = thumb_dict_str.replace("\'", "\"")
        thumb_dict = json.loads(thumb_dict_str)
        prof_pic_url = thumb_dict['default']['url']
        chan_result_dict = {
            'Channel Name':chan_result['title'],
            'Chan_url':'https://www.youtube.com/channel/'+chan_id,
            'Comment Fraction':human_format(count),
            'No. Subscribers':human_format(chan_result['subscriberCount']),
            'No. Views':human_format(chan_result['viewCount_chan']),
            'Likes/Views (on video)':round(chan_result['likeCount'] / chan_result['viewCount_vid'],5),
            'Comments/Views (on video)':round(chan_result['commentCount_vid'] / chan_result['viewCount_vid'],5),
            'Chan Profile Pic':prof_pic_url
            }
        chan_dict_list.append(chan_result_dict)
    
    comm_dict_list= [len(results_info_df)]
    
    for i in results_info_df[0:6].index:
        comm_result_dict = {
        'Name':results_info_df.iloc[i]['authorDisplayName'],
        'Comment':results_info_df.iloc[i]['textDisplay'],
        'Sim Score':round(results_info_df.iloc[i]['sim_score'],2),
        'Profile Pic':results_info_df.iloc[i]['authorProfileImageUrl'],
        'Vid_url':'https://www.youtube.com/watch?v='+results_info_df.iloc[i]['VidID'],
        'Vid_title':results_info_df.iloc[i]['VidTitle'],
        'Channel Name':results_info_df.iloc[i]['title'],
        'Chan_url':'https://www.youtube.com/channel/'+results_info_df.iloc[i]['ChannelID']        
            }
        comm_dict_list.append(comm_result_dict)

    return chan_dict_list, comm_dict_list

In [122]:
chan_res, comm_res = get_comment_channel_results('i am worried about the corona virus', 200)

In [123]:
chan_res

['62',
 {'Channel Name': 'Doctor Mike',
  'Chan_url': 'https://www.youtube.com/channel/UC0QHWhjbe5fGJEPz3sVb6nw',
  'Comment Fraction': '23',
  'No. Subscribers': '4.9M',
  'No. Views': '416M',
  'Likes/Views (on video)': 0.06229,
  'Comments/Views (on video)': 0.01033,
  'Chan Profile Pic': 'https://yt3.ggpht.com/a/AGF-l78Ou_JIdBn62MO19mRg4_ZTSUD-ijfZEKm0QQ=s88-c-k-c0xffffffff-no-rj-mo'},
 {'Channel Name': 'Hyram',
  'Chan_url': 'https://www.youtube.com/channel/UC2sYit3cZ2CuD_8FHYH7O_Q',
  'Comment Fraction': '8',
  'No. Subscribers': '752K',
  'No. Views': '32.2M',
  'Likes/Views (on video)': 0.04057,
  'Comments/Views (on video)': 0.01344,
  'Chan Profile Pic': 'https://yt3.ggpht.com/a/AGF-l79XP96XjoRc1C3H88yyydAsKHYKV4Thuc-LxQ=s88-c-k-c0xffffffff-no-rj-mo'},
 {'Channel Name': 'Cody Ko',
  'Chan_url': 'https://www.youtube.com/channel/UCfp86n--4JvqKbunwSI2lYQ',
  'Comment Fraction': '8',
  'No. Subscribers': '4.3M',
  'No. Views': '634M',
  'Likes/Views (on video)': 0.08029,
  'Comme

#### Basically have the comment functionality all set now, just need to figure out a way to get the new metric in:

In [151]:
from itertools import islice

def take(n, iterable):
    # Return first n items of the iterable as a list
    return list(islice(iterable, n))

def get_comment_channel_results(search_term, num_comms):
    #### Put in search term of interest:
    search_doc = nlp(search_term)
    
    # Get the vector representation:
    search_vec = search_doc.vector.reshape(1,-1)
    
    #### Create similarity vector
    sim_vec = cosine_similarity(search_vec,database_mat)
    
    # Assign the scores to the dataframe and sort by the similarity scores
    comms_df['sim_score'] = (sim_vec[0])

    rel_comms_df = comms_df.sort_values(by='sim_score',ascending=False).head(num_comms)
     
    # Create a results_info_df that collects all the relevant information on 
    # the resultant df:
    results_info_df = rel_comms_df[[
                                    'CommID',
                                    'authorChannelUrl',
                                    'authorDisplayName',
                                    'authorProfileImageUrl',
                                    'parentId',
                                    'publishedAt',
                                    'textDisplay',
                                    'videoId',
                                    'sim_score']].copy(deep=True)
    
    #### Need to rename the column with the video ID to match up with that in the vids_df
    cols = list(results_info_df.columns)
    cols[-2] = 'VidID'
    results_info_df.columns = cols
    results_info_df = results_info_df.merge(
        right=vids_df,
        how='left',
        on='VidID',
        suffixes=('_comm', '_vid'))
    
    # Remove duplicated comments:
    results_info_df.drop_duplicates(subset='CommID', inplace=True)
    
    #### Lastly get the channel info:
    
    # Rename common columns:
    cols = list(results_info_df.columns)    
    cols[5] = 'publishedAt_comm'    
    results_info_df.columns = cols    
    results_info_df = results_info_df.merge(
        right=chan_info_df,
        how='left',
        on='ChannelID',
        suffixes=('_vid', '_chan'))
    
    # Remove duplicated comments:
    results_info_df.drop_duplicates(subset='CommID', inplace=True)
    
    # Create a listing of the channels associated with the comments, 
    # in order of sum of the sim_score
    
    sim_sun_dict = {}

    for chanID in set(results_info_df['ChannelID']):
        try:
            sim_sun_dict[chanID] = sum(results_info_df[results_info_df['ChannelID'] == chanID]['sim_score'])
        except:
            print('There was an error in processing the sum of the similarity score')
    
    sim_sun_sorted_dict = {key: val for key, val 
                           in 
                           sorted(sim_sun_dict.items(), 
                                  reverse=True, 
                                  key=lambda item: item[1])}
    
    top6_chans = take(6, sim_sun_sorted_dict.items())
    
    comm_count = Counter(results_info_df['ChannelID'])
    comm_count_list = comm_count.most_common()
    
    chan_dict_list= [human_format(len(sim_sun_sorted_dict))]
    
    
    
    # Send the relevant info to the website:

    for chan_id,count in top6_chans:
        chan_result = results_info_df[results_info_df['ChannelID'] == chan_id].iloc[0]
        thumb_dict_str = chan_result['thumbnails']
        thumb_dict_str = thumb_dict_str.replace("\'", "\"")
        thumb_dict = json.loads(thumb_dict_str)
        prof_pic_url = thumb_dict['default']['url']
        chan_result_dict = {
            'Channel Name':chan_result['title'],
            'Chan_url':'https://www.youtube.com/channel/'+chan_id,
            'Comment Fraction':human_format(count),
            'No. Subscribers':human_format(chan_result['subscriberCount']),
            'No. Views':human_format(chan_result['viewCount_chan']),
            'Likes/Views (on video)':round(chan_result['likeCount'] / chan_result['viewCount_vid'],5),
            'Comments/Views (on video)':round(chan_result['commentCount_vid'] / chan_result['viewCount_vid'],5),
            'Chan Profile Pic':prof_pic_url
            }
        chan_dict_list.append(chan_result_dict)
    
    comm_dict_list= [len(results_info_df)]
    
    for i in results_info_df[0:6].index:
        comm_result_dict = {
        'Name':results_info_df.iloc[i]['authorDisplayName'],
        'Comment':results_info_df.iloc[i]['textDisplay'],
        'Sim Score':round(results_info_df.iloc[i]['sim_score'],2),
        'Profile Pic':results_info_df.iloc[i]['authorProfileImageUrl'],
        'Vid_url':'https://www.youtube.com/watch?v='+results_info_df.iloc[i]['VidID'],
        'Vid_title':results_info_df.iloc[i]['VidTitle'],
        'Channel Name':results_info_df.iloc[i]['title'],
        'Chan_url':'https://www.youtube.com/channel/'+results_info_df.iloc[i]['ChannelID']        
            }
        comm_dict_list.append(comm_result_dict)

    return chan_dict_list, comm_dict_list

In [152]:
chan_dict_list, comm_dict_list = get_comment_channel_results(search_term, num_comms)

In [153]:
chan_dict_list

['12',
 {'Channel Name': 'Peter McKinnon',
  'Chan_url': 'https://www.youtube.com/channel/UC3DkFux8Iv-aYnTRWzwaiBA',
  'Comment Fraction': '20.8',
  'No. Subscribers': '4.27M',
  'No. Views': '300M',
  'Likes/Views (on video)': 0.0503,
  'Comments/Views (on video)': 0.00312,
  'Chan Profile Pic': 'https://yt3.ggpht.com/a/AGF-l7_zHtm1Mcchr3Zq7q518oPcTJh951-wRp9GHQ=s88-c-k-c0xffffffff-no-rj-mo'},
 {'Channel Name': 'Sarah Rae Vlogas',
  'Chan_url': 'https://www.youtube.com/channel/UC0YgscPJVsW-4tIx2U499cw',
  'Comment Fraction': '1.77',
  'No. Subscribers': '252K',
  'No. Views': '36.2M',
  'Likes/Views (on video)': 0.03935,
  'Comments/Views (on video)': 0.00395,
  'Chan Profile Pic': 'https://yt3.ggpht.com/a/AGF-l7-lBZPgpr7QsaBQELzS5Rz62fC7Vr3rVKqKww=s88-c-k-c0xffffffff-no-rj-mo'},
 {'Channel Name': 'joanday',
  'Chan_url': 'https://www.youtube.com/channel/UCAo41QETLlwvJ-oqjmEPt8A',
  'Comment Fraction': '1.16',
  'No. Subscribers': '397K',
  'No. Views': '51.9M',
  'Likes/Views (on vid

Happy with the result now